In [1]:
%defaultDatasource jdbc:h2:mem:db

In [2]:
CREATE TABLE column_3C_weka (
    pelvic_incidence DOUBLE,
    pelvic_tilt DOUBLE,
    lumbar_lordosis_angle DOUBLE,
    sacral_slope DOUBLE,
    pelvic_radius DOUBLE,
    degree_spondylolisthesis DOUBLE,
    class VARCHAR(50)
) AS SELECT
    pelvic_incidence,
    pelvic_tilt,
    lumbar_lordosis_angle,
    sacral_slope,
    pelvic_radius,
    degree_spondylolisthesis,
    class
FROM CSVREAD('../data/column_3C_weka.csv');

In [3]:
SELECT * FROM column_3c_weka

# Modelo Conceitual

### Antes:
![uml-etapa01](../img/uml-etapa01.png)

### Depois:
![uml-etapa02](../img/uml-etapa02.png)

#### Motivo
* Modificação nos atributos da classe Paciente: agora ela possui somente o identificador do paciente chamado idPaciente, os atributos referentes à situação do mesmo (como Incidência Pélvica, Versão Pélvica, etc) foram transferidos para outras classes a fim de que se pudesse, caso surgisse uma nova característica (novo tipo de medição da Pelve, por exemplo) seria possível adicioná-la ao banco;  
* Seguindo a modificação do Paciente, foram adicionadas duas novas classes: Possui e Característica. Cada Paciente possui várias características (N:N), sendo que o valor do relacionamento entre o Paciente e sua Característica fica na classe intermediária Possui - por exemplo, João (Paciente) tem uma Incidência Pélvica (Característica) de valor 60° (Possui) e Maria (Paciente) tem uma Incidência Pélvica (Característica) de 70° (Possui);  
* Mudança no nome da classe Doença para Classificação e tipoDoença para classe, pois havia certos pacientes classificados como “Normal”, o que não caracterizava um problema.  


# Modelo Logico

![modelo-logico](../img/modelo-logico.png)

# Tabelas Relacionais

In [4]:
DROP TABLE IF EXISTS ClassificacaoTemSintoma;
DROP TABLE IF EXISTS PacientePossuiCaracteristica;
DROP TABLE IF EXISTS Caracteristica;
DROP TABLE IF EXISTS Sintoma;
DROP TABLE IF EXISTS Classificacao;
DROP TABLE IF EXISTS Paciente;

CREATE TABLE Caracteristica (
    nomeCaracteristica VARCHAR(50),
    PRIMARY KEY(nomeCaracteristica)
);

CREATE TABLE Sintoma (
    idSintoma INTEGER AUTO_INCREMENT,
    localizacaoCorpo VARCHAR(50),
    descricao VARCHAR(200),
    PRIMARY KEY(idSintoma)
);

CREATE TABLE Classificacao (
    classe VARCHAR(50),
    PRIMARY KEY(classe)
);

CREATE TABLE ClassificacaoTemSintoma (
    IdClassificacaoTemSintoma INTEGER AUTO_INCREMENT,
    idClassificacao INTEGER,
    idSintoma INTEGER,
    PRIMARY KEY(IdClassificacaoTemSintoma),
    FOREIGN KEY(idClassificacao)
        REFERENCES Classificacao(idClassificacao)
            ON DELETE CASCADE
            ON UPDATE CASCADE,
    FOREIGN KEY(idSintoma)
        REFERENCES Sintoma(idSintoma)
          ON DELETE CASCADE
          ON UPDATE CASCADE
);

CREATE TABLE Paciente (
    idPaciente INTEGER AUTO_INCREMENT,
    idClassificacao INTEGER,
    PRIMARY KEY(idPaciente),
    FOREIGN KEY(idClassificacao)
        REFERENCES Classificacao(idClassificacao)
          ON DELETE CASCADE
          ON UPDATE CASCADE
);

CREATE TABLE PacientePossuiCaracteristica (
    idPacienteCaracteristica INTEGER AUTO_INCREMENT, 
    idPaciente INTEGER,
    idCaracteristica INTEGER, 
    valor DOUBLE,
    PRIMARY KEY(idPacienteCaracteristica),
    FOREIGN KEY(idPaciente)
        REFERENCES Paciente(idPaciente)
          ON DELETE CASCADE
          ON UPDATE CASCADE,
    FOREIGN KEY(idCaracteristica)
        REFERENCES Caracteristica(idCaracteristica)
          ON DELETE CASCADE
          ON UPDATE CASCADE
);

SELECT * FROM Paciente;
SELECT * FROM PacientePossuiCaracteristica;
SELECT * FROM Caracteristica;
SELECT * FROM Classificacao;
SELECT * FROM ClassificacaoTemSintoma;
SELECT * FROM Sintoma;

org.h2.jdbc.JdbcSQLException:  Column "IDCLASSIFICACAO" not found; SQL statement

# Inserção dos dados no modelo lógico

In [10]:
DROP TABLE IF EXISTS Paciente;
DROP TABLE IF EXISTS Classificacao;

CREATE TABLE Paciente (
    nomeClassificacao VARCHAR(50)
) AS SELECT
    class
FROM column_3C_weka;

ALTER TABLE Paciente ADD idPaciente INTEGER AUTO_INCREMENT; 
ALTER TABLE Paciente ADD CONSTRAINT pk_Paciente PRIMARY KEY (idPaciente);

SELECT * FROM Paciente;

CREATE TABLE Classificacao (
    idClassificacao INTEGER AUTO_INCREMENT,
    classe VARCHAR(50),
    PRIMARY KEY(classe)
);

In [14]:
INSERT INTO Classificacao(classe)
SELECT DISTINCT NomeClassificacao FROM Paciente
SELECT * FROM Classificacao;

org.h2.jdbc.JdbcSQLException:  Syntax error in SQL statement "INSERT INTO CLASSIFICACAO(CLASSE)

# Querys Criativas